In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from bs4 import BeautifulSoup 
import urllib
import urllib.request as rq
import pandas as pd

global news_df

def News():
    global news_df
    
    keyword = input('검색 키워드를 입력해주세요 ')
    keyword = urllib.parse.quote(keyword) #퍼센트인코딩 #URL에 문자를 표현하는 인코딩방법(URL에 한글이 섞이면 오류 발생)
    num = int(input('추출할 뉴스기사의 숫자를 입력해주세요 '))
    
    news_df = pd.DataFrame(columns=['title','url']) 
    
    page_num=1
    i=0 #크롤링한 기사의 수
    while num > i:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+keyword+'&start='+str(page_num)
        html = rq.urlopen(url)
        bs = BeautifulSoup(html,'html.parser') ##html_doc 데이터를 html.parser로 파싱한 뒤 BeautifulSoup 객체를 생성해서 bs 변수에 넣음 
        texts = bs.find_all(class_='news_tit')

        for text in texts:
            news_df.loc[i,'title'] = text.get_text()
            news_df.loc[i,'url'] = text.get('href')
            # news_df.loc[i,'url'] = text['href']
            i +=1
            if i == num:
                break
        page_num +=10
    return news_df

News()

검색 키워드를 입력해주세요  공공도서관
추출할 뉴스기사의 숫자를 입력해주세요  1000


,title,url
0,"서구 공공도서관, 독서의 달 행사 운영",http://www.dtnews24.com/news/articleView.html?...
1,"""작은도서관 서비스 개선에 큰 도움"" 대학생 공공기관 지원활동 호평",http://www.kado.net/news/articleView.html?idxn...
2,"남양주시 와부도서관, '공공도서관 문화가 있는 날' 공연",https://www.gukjenews.com/news/articleView.htm...
3,"사천시의회, 전주 공공도서관 건립 우수사례 벤치마킹",http://www.newsfreezone.co.kr/news/articleView...
4,"사천시의회, 공공도서관 건립 우수사례 벤치마킹 실시",https://www.pressian.com/pages/articles/202208...
...,...,...
995,"슬기샘어린이도서관, 공공디자인 대상 수상...어떤 모습일까?",http://www.babytimes.co.kr/news/articleView.ht...
996,[이제는 지방시대] 독서 공간·놀이터 그리고 쉼터… 전주 도서관들 명소 됐어...,https://news.kmib.co.kr/article/view.asp?arcid...
997,"부산시, 부전도서관 공공개발방안 시민과 함께 만든다!",http://www.enewstoday.co.kr/news/articleView.h...
998,3년간 공공도서관 대출 1위 도스토옙스키 작품은 ‘죄와 벌’,https://news.kbs.co.kr/news/view.do?ncd=534258...


In [ ]:
import re
target = news_df['title'].values #Series.values = array
title_list = target.tolist() # array > list
text = ' '.join(title_list) #list들을 공백으로 구분해서 연결
text = re.sub('[^가-힣]+',' ',text) #한글 이외는 공백으로 대체

In [ ]:
#konlpy(콘엘피) = 형태소 분석
from konlpy.tag import Okt
from collections import Counter
okt = Okt() #객체형성

morph = okt.pos(text) #형태소로 변환 #word= 단어 / tag=품사
morph

noun_list =[]
stop_word = []
for word, tag in morph: #word= 단어 / tag=품사
    if(tag=='Noun') & (len(word)>1): #noun이고 단어가 1글자 이상인것(1글자는 보통 조사이기때문)
        if word not in stop_word:
            noun_list.append(word)

count = Counter(noun_list) #단어별 숫자세주기


data =dict(count.most_common()) #빈도수 정리 내림차
series = pd.Series(data)
df=pd.DataFrame(series, columns=['cnt'])
# lib2022 =df.loc['청소년'].cnt

df